# Etapa 1: Importação das bibliotecas

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 502.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30509 sha256=589b8cc12fb5241b2fa120ab9af963d5e58c1149edf2fe6473af27bcbc7fccc1
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19454 sha256=985f2ca6a7fda58ace7b5d3a0a85474b031f9149352034c1bd13c5886cdee774
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=b6c53255c1ebdaaa59ed21cace2972d93d84ec94d459584a9b61350ae90521ab
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e9087

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
tf.__version__
# !pip install tensorflow==2.2.0-rc3

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


'2.15.0'

# Etapa 2: Pré-processamento

## Carregamento dos arquivos

In [4]:
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/Base de dados sentimentos/training.1600000.processed.noemoticon.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [7]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [8]:
data.shape

(1600000, 2)

In [9]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Pré-processamento

### Limpeza

In [10]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [11]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

<ipython-input-10-6bb4805d14d5>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tweet = BeautifulSoup(tweet, "lxml").get_text()


In [12]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenização

In [13]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [14]:
def encode_sentence(sent):
  return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [15]:
encode_sentence("My dog likes strawberries.")

['[CLS]', 'my', 'dog', 'likes', 'straw', '##berries', '.', '[SEP]']

In [17]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [18]:
print(data_inputs[0:2])

[['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]'], ['[CLS]', 'is', 'upset', 'that', 'he', 'can', "'", 't', 'update', 'his', 'facebook', 'by', 'text', '##ing', 'it', '.', '.', '.', 'and', 'might', 'cry', 'as', 'a', 'result', 'school', 'today', 'also', '.', 'blah', '!', '[SEP]']]


### Criação da base de dados

In [19]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

In [20]:
get_ids(tokenizer.tokenize("My dog likes strawberries."))

[2026, 3899, 7777, 13137, 20968, 1012]

In [21]:
np.char.not_equal("[PAD]", "[PAD]")

array(False)

In [22]:
def get_mask(tokens):
  return np.char.not_equal(tokens, "[PAD]").astype(int)

In [23]:
get_mask(tokenizer.tokenize("My dog likes strawberries."))

array([1, 1, 1, 1, 1, 1])

In [24]:
def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [25]:
print(data_inputs[0])

['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]']


In [29]:
get_segments(data_inputs[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
my_sent

['[CLS]', 'roses', 'are', 'red', '.', '[SEP]']

In [31]:
bert_layer([
            tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0)
           ])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.27935600e-01, -4.10335541e-01, -9.65755045e-01,
          9.07317877e-01,  8.12913954e-01, -1.74174443e-01,
          9.11234438e-01,  3.41952205e-01, -8.74521315e-01,
         -9.99989271e-01, -7.78410077e-01,  9.69385386e-01,
          9.86160517e-01,  6.36963367e-01,  9.48631227e-01,
         -7.51193225e-01, -4.58339423e-01, -7.08104610e-01,
          4.62098390e-01, -6.57927155e-01,  7.60414600e-01,
          9.99994814e-01, -3.96861017e-01,  3.44166100e-01,
          6.16488695e-01,  9.94400144e-01, -7.76633859e-01,
          9.38316524e-01,  9.59452271e-01,  7.32879341e-01,
         -6.93436742e-01,  2.93080539e-01, -9.93785560e-01,
         -1.64551765e-01, -9.67019677e-01, -9.95549619e-01,
          5.32935560e-01, -6.88060939e-01,  1.34715773e-02,
          2.98194103e-02, -9.18356538e-01,  4.20526266e-01,
          9.99988973e-01,  2.52676845e-01,  6.06235564e-01,
         -3.50750148e-01, -1.00000000e+00,  4.975

In [32]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
               get_mask(sent_lab[0]),
               get_segments(sent_lab[0])],
              sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [36]:
sorted_all[0]

([[101, 4604, 2033, 1037, 4957, 1045, 3335, 102],
  array([1, 1, 1, 1, 1, 1, 1, 1]),
  [0, 0, 0, 0, 0, 0, 0, 0]],
 0)

In [33]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [34]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [35]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Etapa 3: Construção do modelo

In [37]:
class DCNNBERTEmbedding(tf.keras.Model):

    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)

        self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                                         trainable = False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")

    def embed_with_bert(self, all_tokens):
      _, embs = self.bert_layer([all_tokens[:, 0, :],
                                 all_tokens[:, 1, :],
                                 all_tokens[:, 2, :]])
      return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

# Etapa 4: Treinamento

In [38]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 5

In [39]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [40]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [45]:
checkpoint_path = "/content/drive/My Drive/Cursos - recursos"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [42]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
history = Dcnn.fit(train_dataset,
                   epochs=NB_EPOCHS,
                   callbacks=[MyCustomCallback()])

Epoch 1/5
  40623/Unknown - 2999s 74ms/step - loss: 0.3973 - accuracy: 0.8219Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 3001s 74ms/step - loss: 0.3973 - accuracy: 0.8219
Epoch 2/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3756 - accuracy: 0.8339Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2987s 74ms/step - loss: 0.3756 - accuracy: 0.8339
Epoch 3/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3659 - accuracy: 0.8391Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2979s 73ms/step - loss: 0.3659 - accuracy: 0.8391
Epoch 4/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3586 - accuracy: 0.8426Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2969s 73ms/step - loss: 0.3586 - accuracy: 0.8426
Epoch 5/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3527 - accuracy: 0.8454Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2960s 73ms/step - loss: 0.3527 - accuracy: 0.8454
<tensorflow.python.keras.callbacks.History at 0x7fd363ef4080>

# Etapa 5: Avaliação do modelo

In [46]:
results = Dcnn.evaluate(test_dataset)
print(results)

4513/4513 [==============================] - 116s 25ms/step - loss: 0.3769 - accuracy: 0.8384
[0.37687262892723083, 0.8383697271347046]


In [47]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)

  input_ids = get_ids(tokens)
  input_mask = get_mask(tokens)
  segment_ids = get_segments(tokens)

  inputs = tf.stack(
      [
       tf.cast(input_ids, dtype=tf.int32),
       tf.cast(input_mask, dtype=tf.int32),
       tf.cast(segment_ids, dtype=tf.int32),
      ], axis = 0)
  inputs = tf.expand_dims(inputs, 0)

  output = Dcnn(inputs, training=False)

  sentiment = math.floor(output*2)

  if sentiment == 0:
    print("Output of the model: {}\nPredicted sentiment: negative".format(output))
  elif sentiment == 1:
    print("Output of the model: {}\nPredicted sentiment: positive".format(output))


In [48]:
get_prediction("This actor is very bad.")

Output of the model: [[0.06391602]]
Predicted sentiment: negative


In [49]:
get_prediction("I like dogs.")

Output of the model: [[0.88880855]]
Predicted sentiment: positive
